In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from scipy.io import mmread, mmwrite
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
# from scipy import sparse
import plotly.express as px
from scipy import sparse
import png
import math
import scipy.linalg as LA
import nodevectors
from sklearn.decomposition import PCA
import seaborn as sns
from tabulate import tabulate
from scipy.stats.mstats import winsorize

In [3]:
ladRegion_lookup = pd.read_csv("LAD_Region_lookup_GB.csv")

In [7]:
# reset the index so it is what i want it to be
ladRegion_lookup['index'] = ladRegion_lookup.index

,Unnamed: 0,index,LAD,LAD11CD,LAD11NM,RGN11CD,RGN11NM
0,0,0,St Albans,E07000100,St Albans,E12000006,East of England
1,1,1,Welwyn Hatfield,E07000104,Welwyn Hatfield,E12000006,East of England
2,2,2,Hertsmere,E07000098,Hertsmere,E12000006,East of England
3,3,3,Dacorum,E07000096,Dacorum,E12000006,East of England
4,4,4,Central Bedfordshire,E06000056,Central Bedfordshire,E12000006,East of England
...,...,...,...,...,...,...,...
375,27,375,South Ayrshire,S12000028,South Ayrshire,S92000003,Scotland
376,28,376,South Lanarkshire,S12000029,South Lanarkshire,S92000003,Scotland
377,29,377,Stirling,S12000030,Stirling,S92000003,Scotland
378,30,378,West Dunbartonshire,S12000039,West Dunbartonshire,S92000003,Scotland


In [9]:
regs = ladRegion_lookup['RGN11NM'].unique()
regs

array(['East of England', 'West Midlands', 'East Midlands', 'South West',
       'North West', 'Yorkshire and The Humber', 'South East', 'London',
       'North East', 'Wales', 'Scotland'], dtype=object)

In [11]:
A_list = []
A_RegReg_list = []
year_list = []

for year in range(2005,2011):
    # does 2005-2010 inclusive
    filepath = "A_LAD_" + str(year) + "_UK_v5.mtx" # file name to read in 
    A = mmread(filepath) # temporary
    
    year_list.append(year)
    
    A = A.toarray()
    
    A_RegLAD = list()
    A_RegReg =list() # empty numpy array to fill, one row
    # to make "RegxReg" version of A 

    for reg in regs: 
        
        # find all the indexes for the given reg
        row_indexes = ladRegion_lookup.loc[ladRegion_lookup["RGN11NM"] == reg, "index"]
        # make a numpy array and subtract 1, as python indexes from 0 not 1
        row_indexes  = row_indexes.to_numpy()
        row_indexes = [x - 1 for x in row_indexes]
        column_indexes = row_indexes
        
        # find all the rows with the indexes for that region, and add them together to make one row 
        row = A[row_indexes, :]
        row = np.sum(row, axis=0)
        
        # one row for each region in the matrix A_RegLAD
        A_RegLAD.append(row)
        
    A_RegLAD = np.asarray(A_RegLAD)
    
    for reg in regs: 
        
        # find all the columns with the indexes for that region, from the already row summed matrix, 
        # and add them together to make one column 
        column_indexes = ladRegion_lookup.loc[ladRegion_lookup["RGN11NM"] == reg, "index"]
        column_indexes = column_indexes.to_numpy()
        column_indexes = [x - 1 for x in column_indexes]

        column = A_RegLAD[:, column_indexes]
        column = np.sum(column, axis=1)
        
        A_RegReg.append(column)
    
    A_RegReg = np.asarray(A_RegReg)
    A_RegReg = sparse.coo_matrix(A_RegReg)

    # add a matrix for each year to the list
    A_RegReg_list.append(A_RegReg)
    
#print(A_RegMSOA_list)    
A_RegReg_all = hstack(A_RegReg_list)

In [13]:
# save all the Reg x Reg matrices

for i in range(0,6):
    mat = A_RegReg_list[i]
    mmwrite("A_Reg_" + str(i+2005) + "_GB.mtx", mat)